In [1]:
import geopandas as gpd
import pyarrow
import pandas as pd

In [17]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [18]:
target_2631.head()

,utc_timestamp,caid,geo_hash,latitude,longitude,utc_datetime,la_datetime,hour,lat_epsg_3857,lon_epsg_3857
0,1572117649,0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e3...,9qbdwqx8v,38.444053,-122.76167,2019-10-26 19:20:49+00:00,2019-10-26 12:20:49-07:00,12,-1.366577e+07,4.642347e+06
1,1572142983,0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e3...,9qbdwqx8v,38.444056,-122.76167,2019-10-27 02:23:03+00:00,2019-10-26 19:23:03-07:00,19,-1.366577e+07,4.642347e+06
2,1572180736,0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e3...,9qbdwwdxk,38.445193,-122.75760,2019-10-27 12:52:16+00:00,2019-10-27 05:52:16-07:00,5,-1.366531e+07,4.642509e+06
3,1572216715,0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e3...,9r4besxm0,39.488493,-119.71904,2019-10-27 22:51:55+00:00,2019-10-27 15:51:55-07:00,15,-1.332706e+07,4.791888e+06
4,1572171617,000240069cae6c8106ed10b34ac23e734f7e156a9e170d...,9qbdpsyqx,38.347546,-122.70838,2019-10-27 10:20:17+00:00,2019-10-27 03:20:17-07:00,3,-1.365983e+07,4.628639e+06


In [19]:
import os
def write_file(filename, info):
  text_file = open(filename, "w")
  text_file.write(info)
  text_file.close()

In [ ]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
print("start index: %d, caid: %s" %(0, start_id))
end = 200000
for index in range(0, end):
  data_id = target_2631.loc[index]['caid']
  if data_id != start_id:
    print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    print("total points: %d" %(index - start_index))
    start_index = index
    start_id = data_id
    print("start index: %d, caid: %s" %(index, data_id))
print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))



In [ ]:
import geopandas as gpd
res = 'LINESTRING('
df = pd.DataFrame(
    {'Latitude': [float(target_2631.loc[2797]['latitude'])],
     'Longitude': [float(target_2631.loc[2797]['longitude'])]})

#for index in range_:
start_index = 534
end_index = 2781
for index in range(start_index,end_index):
  
  single_data = target_2631.loc[index]
  if ((single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)):
    lat = float(single_data['latitude'])
    lon = float(single_data['longitude'])
    df.loc[len(df.index)] = [float(lat),float(lon)] 
    res+= (str(lon) + ' ')
    if index != end_index - 1:
      res+= (str(lat) + ',')
    else:
      res+= (str(lat))
res+=')'
df = df[1:-1]
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
write_file('data/GPS.nosync/output/' + target_2631.loc[start_index]['caid']+'-points.txt', res)

In [ ]:
len(gdf)

In [ ]:
streets = gpd.read_file('data/GPS.nosync/Streets/Streets.shp')

In [ ]:
file_path1 = "data/GPS.nosync/output/test002-m_wkt.txt"
file_path2 = "data/GPS.nosync/output/test002-e_wkt.txt"
#check if file is present
if os.path.isfile(file_path1):
    #open text file in read mode
    text_file = open(file_path1, "r")
 
    #read whole file to a string
    wkt1 = text_file.read()
 
    #close file
    text_file.close()
if os.path.isfile(file_path2):
    #open text file in read mode
    text_file = open(file_path2, "r")
 
    #read whole file to a string
    wkt2 = text_file.read()
 
    #close file
    text_file.close()
d1 = {'col1': ['name1', 'name2'], 'wkt': wkt1}
d2 = {'col1': ['name1', 'name2'], 'wkt': wkt2}
df = pd.DataFrame(d1)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf1 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
df = pd.DataFrame(d2)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf2 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
streets.plot(ax=ax, color='black', LineWidth = 1)
#gdf2.plot(ax=ax, color='green',LineWidth = 2)
gdf.plot(ax=ax, color='blue', LineWidth = 2)
#gdf1.plot(ax=ax, color='red',LineWidth = 2)

In [111]:
parts = streets[1:10000]
parts['points'] = parts.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
len(parts)

/Users/xiongruoyang/miniconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


9999

In [ ]:
import folium
map = folium.Map(location = [4,10], tiles = "Stamen Terrain", zoom_start = 3)

In [ ]:
from folium import plugins

map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)

#heat_data = [gdf2.points[0],gdf2.points[1]]
#heat_data[1] = 
heat_data = [[point[1], point[0]] for point in gdf2['points'][0] ]
for point in gdf2['points'][1]:
  heat_data.append([point[1],point[0]])
for points in parts['points']:
  for p in points:
    heat_data.append([p[1],p[0]])
heat_data
plugins.HeatMap(heat_data).add_to(map)
map


In [113]:
map.save("index.html")

In [ ]:
plugins.HeatMap(heat_data).add_to(map)
map